In [13]:
import torch
import pandas as pd
from PIL import Image

#data format setup:
torch.set_default_tensor_type('torch.cuda.FloatTensor')
input_data = torch.tensor([0,0,0,0,0,0,0,0,0], dtype=torch.float)

def play_ground(h, w, agent_a, agent_b, agent_c, agent_d, ball):
    rows = h
    columns = w * 3 + 2
    init_ground = torch.tensor([[0 for x in range(columns)] for y in range(rows)], dtype=torch.float)
    for i in range(0, rows):
        for j in [w, -w - 1]:
            init_ground[i][j] = 1

    for i in [agent_a[0]]:
        for j in range(agent_a[1] + w, agent_a[3] + 1 + w):
            init_ground[i][j] = 1
    for i in [agent_b[0]]:
        for j in range(agent_b[1] + w, agent_b[3] + 1 + w):
            init_ground[i][j] = 1
    for i in [agent_c[0]]:
        for j in range(agent_c[3] + w, agent_c[1] + 1 + w):
            init_ground[i][j] = 1
    for i in [agent_d[0]]:
        for j in range(agent_d[3] + w, agent_d[1] + 1 + w):
            init_ground[i][j] = 1
    init_ground[ball[0]][ball[1]] = 2
    return init_ground

def agent_ball(perception, state):
    rows = 8
    columns = 3
    reference_matrix = torch.tensor([[0 for x in range(columns)] for y in range(rows)], dtype=torch.int)
    reference_matrix[0][1] = -1
    reference_matrix[0][2] = -1
    reference_matrix[1][1] = -1
    reference_matrix[1][2] = 0
    reference_matrix[2][1] = -1
    reference_matrix[2][2] = 1
    reference_matrix[3][1] = 0
    reference_matrix[3][2] = 1
    reference_matrix[4][1] = 1
    reference_matrix[4][2] = 1
    reference_matrix[5][1] = 1
    reference_matrix[5][2] = 0
    reference_matrix[6][1] = 1
    reference_matrix[6][2] = -1
    reference_matrix[7][1] = 0
    reference_matrix[7][2] = -1

    if (state[2] * state[3]) == 0:
        if perception == [1, 1, 1]:
            new_speed_x = -state[2]
            new_speed_y = -state[3]
        elif perception == [0, 1, 1]:
            for i in range(0,8):
                if (state[2] == reference_matrix[i][1]) & (state[3] == reference_matrix[i][2]):
                    start_point = i
                    print(perception)
                    print(start_point)
            new_speed_x = reference_matrix[start_point - 2][1]
            new_speed_y = reference_matrix[start_point - 2][2]
        elif perception == [1, 1, 0]:
            for i in range(0,8):
                if (state[2] == reference_matrix[i][1]) & (state[3] == reference_matrix[i][2]):
                    start_point = i
            new_speed_x = reference_matrix[(start_point + 2)% 8][1]
            new_speed_y = reference_matrix[(start_point + 2)% 8][2]
        elif perception == [0, 1, 0]:
            new_speed_x = -state[2]
            new_speed_y = -state[3]
        else:
            new_speed_x = state[2]
            new_speed_y = state[3]
    else:
        if (perception[0] == 1) & (perception[2] == 1):
            new_speed_x = -state[2]
            new_speed_y = -state[3]
        elif perception == [0, 1, 0]:
            new_speed_x = -state[2]
            new_speed_y = -state[3]
        elif perception[2] == 1:
            for i in range(0,8):
                if (state[2] == reference_matrix[i][1]) & (state[3] == reference_matrix[i][2]):
                    start_point = i
                    print(perception)
                    print(start_point)
            new_speed_x = reference_matrix[start_point - 2][1]
            new_speed_y = reference_matrix[start_point - 2][2]
        elif perception[0] == 1:
            for i in range(0,8):
                if (state[2] == reference_matrix[i][1]) & (state[3] == reference_matrix[i][2]):
                    start_point = i
            new_speed_x = reference_matrix[(start_point + 2)% 8][1]
            new_speed_y = reference_matrix[(start_point + 2)% 8][2]
        else:
            new_speed_x = state[2]
            new_speed_y = state[3]

    action = [new_speed_x, new_speed_y]
    state = [state[0] + new_speed_x, state[1] + new_speed_y, new_speed_x, new_speed_y]
    return (action, state)


def func_perspection(play_ground, state):
    rows = 8
    columns = 3
    reference_matrix = torch.tensor([[0 for x in range(columns)] for y in range(rows)], dtype=torch.int)
    reference_matrix[0][1] = -1
    reference_matrix[0][2] = -1
    reference_matrix[1][1] = -1
    reference_matrix[1][2] = 0
    reference_matrix[2][1] = -1
    reference_matrix[2][2] = 1
    reference_matrix[3][1] = 0
    reference_matrix[3][2] = 1
    reference_matrix[4][1] = 1
    reference_matrix[4][2] = 1
    reference_matrix[5][1] = 1
    reference_matrix[5][2] = 0
    reference_matrix[6][1] = 1
    reference_matrix[6][2] = -1
    reference_matrix[7][1] = 0
    reference_matrix[7][2] = -1
    for i in range(0,8):
        reference_matrix[i][0] = play_ground[state[0] + reference_matrix[i][1]][state[1] + reference_matrix[i][2]]
        if (state[2] == reference_matrix[i][1]) & (state[3] == reference_matrix[i][2]):
            start_point = i
    #print(reference_matrix)
    if (state[2] * state[3]) == 0:
        new_perception = [reference_matrix[start_point - 2][0], reference_matrix[start_point][0], reference_matrix[(start_point + 2)% 8][0]]
    else:
        new_perception = [reference_matrix[start_point - 1][0], reference_matrix[start_point][0], reference_matrix[(start_point + 1)% 8][0]]
    #print(new_perception)
    return new_perception

def agent_a(perception, state):
    action = 0
    state = 0
    return (action, state)



if __name__ == "__main__":
    #print(play_ground(12,10,[-1, 1, -1, 3],[-1, 4, -1, 6], [0, 6, 0, 4], [0, 3, 0, 1], [3, 3]))
    #New_play_ground = play_ground(12,10,[-1, 1, -1, 3],[-1, 4, -1, 6], [0, 6, 0, 4], [0, 3, 0, 1], [3, 3])
    #print(agent_ball([1, 0, 0], [5, 15, 1, 1]))
    current_state = [5, 16, 0, -1]
    #New_action, New_state = agent_ball([0, 0, 0], [5, 16, 1, 1])
    for i in range(0,15):
        Current_play_ground = play_ground(12,10,[-1, 1, -1, 3],[-1, 4, -1, 6], [0, 6, 0, 4], [0, 3, 0, 1], current_state[0: 2])
        #print(Current_play_ground)
        Current_perception = func_perspection(Current_play_ground, current_state)
        New_action, New_state = agent_ball(Current_perception, current_state)
        current_state = New_state
        #print(New_action)
        #print(New_state)
        #print(new_perception)
        
        #print(New_play_ground)
        export_output = Current_play_ground.clone()
        export_output = export_output.cpu().detach().numpy()
        export_output = export_output * 120
        img = Image.fromarray(export_output)
        #img.save('my.png')
        img.show()
    #print(func_perspection(New_play_ground, New_state))
    


NameError: name 'reference_matrix' is not defined